In [1]:
from hope.models.segmodel import SegModel
import torch
from hope.utils.embed import create_embs_from_names
import cv2
from hope.utils.transform_utils import resize_by_scaled_short_side

/home/penguin/miniconda3/envs/research/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
model = SegModel(criterions=None,
                     num_classes=512,
                     load_imagenet_model=False, imagenet_ckpt_fpath='')

In [3]:
model.eval()

SegModel(
  (segmodel): SegFormer(
    (encoder): mit_b5(
      (patch_embed1): OverlapPatchEmbed(
        (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed2): OverlapPatchEmbed(
        (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed3): OverlapPatchEmbed(
        (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed4): OverlapPatchEmbed(
        (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (block1): ModuleList(
        (0): Block(
          (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
          (attn

In [4]:
model = torch.nn.DataParallel(model)

In [5]:
checkpoint = torch.load('/home/penguin/SSIW/data/model/segformer_7data.pth', map_location='cpu')['state_dict']
ckpt_filter = {k: v for k, v in checkpoint.items() if 'criterion.0.criterion.weight' not in k}
maybe_errors = model.load_state_dict(ckpt_filter, strict=False)

In [6]:
model = model.to('cuda')

In [7]:
import json

In [8]:
with open('/home/penguin/SSIW/Test_Minist/cmp_dataset_desc.json', 'r', encoding='utf-8') as f:
    new_definitions = json.load(f)

In [9]:
user_label = ['background','facade','window','door', 'cornice', 'sill', 'balcony', 'blind','deco', 'molding', 'pillar', 'shop', 'unlabel']


In [10]:
gt_embs_list = create_embs_from_names(user_label, new_definitions).float()

In [11]:
rgb = cv2.imread('/home/penguin/SSIW/data/base/cmp_b0001.jpg', -1)[:, :, ::-1]

In [12]:
image_resized = resize_by_scaled_short_side(rgb, 720, 1)
h, w, _ = rgb.shape

In [13]:
from hope.utils.transform_utils import single_scale_single_crop_cuda

In [14]:
out_logit, emb = single_scale_single_crop_cuda(model, image_resized, h, w, gt_embs_list=gt_embs_list, args=None)

IMAGE MODEL EMBED:  tensor([[[[ 0.9677,  0.9555,  0.9432,  ...,  0.4785,  0.4577,  0.4370],
          [ 1.0237,  0.9959,  0.9681,  ...,  0.5186,  0.4955,  0.4723],
          [ 1.0796,  1.0363,  0.9930,  ...,  0.5588,  0.5332,  0.5076],
          ...,
          [ 1.4984,  1.4334,  1.3684,  ...,  1.2053,  1.2261,  1.2469],
          [ 1.5373,  1.5072,  1.4771,  ...,  1.2232,  1.2122,  1.2012],
          [ 1.5762,  1.5810,  1.5858,  ...,  1.2411,  1.1983,  1.1556]],

         [[ 1.1118,  1.0646,  1.0175,  ...,  2.6556,  2.6401,  2.6246],
          [ 1.2082,  1.1665,  1.1248,  ...,  2.7158,  2.7480,  2.7803],
          [ 1.3045,  1.2683,  1.2320,  ...,  2.7760,  2.8560,  2.9359],
          ...,
          [ 3.0590,  2.9372,  2.8154,  ...,  2.1309,  2.2640,  2.3971],
          [ 3.1216,  3.0577,  2.9939,  ...,  2.1646,  2.2413,  2.3181],
          [ 3.1841,  3.1782,  3.1723,  ...,  2.1983,  2.2186,  2.2390]],

         [[ 5.6221,  5.5498,  5.4774,  ...,  7.1191,  7.0593,  6.9995],
          

In [16]:
out_logit

array([[[7.30377680e-04, 7.89794028e-01, 7.67565425e-03, ...,
         6.36376217e-02, 7.25746527e-02, 6.25764718e-04],
        [7.74360436e-04, 7.97130048e-01, 6.53548725e-03, ...,
         7.63333440e-02, 6.15738556e-02, 6.96829404e-04],
        [8.22321395e-04, 7.98902333e-01, 5.60233276e-03, ...,
         9.12014246e-02, 5.22993207e-02, 7.74080167e-04],
        ...,
        [3.27872316e-04, 9.11018610e-01, 4.28098254e-03, ...,
         3.81366424e-02, 2.08857302e-02, 3.42788117e-04],
        [2.94951111e-04, 9.11164701e-01, 4.30622045e-03, ...,
         3.53606194e-02, 2.25036740e-02, 2.89288757e-04],
        [2.64814385e-04, 9.10337687e-01, 4.35221940e-03, ...,
         3.28608938e-02, 2.44434718e-02, 2.42088572e-04]],

       [[1.12461124e-03, 7.12043285e-01, 1.09147439e-02, ...,
         1.31265298e-01, 8.20370466e-02, 7.01031182e-04],
        [1.17005152e-03, 7.27108717e-01, 7.71105802e-03, ...,
         1.54981747e-01, 6.40132129e-02, 7.93138053e-04],
        [1.21196837e-03, 

In [15]:
emb

tensor([[[[ 0.0174,  0.0173,  0.0172,  ...,  0.0073,  0.0070,  0.0067],
          [ 0.0181,  0.0179,  0.0177,  ...,  0.0079,  0.0076,  0.0072],
          [ 0.0187,  0.0185,  0.0182,  ...,  0.0086,  0.0081,  0.0076],
          ...,
          [ 0.0175,  0.0172,  0.0167,  ...,  0.0128,  0.0129,  0.0129],
          [ 0.0178,  0.0175,  0.0172,  ...,  0.0126,  0.0126,  0.0126],
          [ 0.0180,  0.0178,  0.0177,  ...,  0.0125,  0.0124,  0.0123]],

         [[ 0.0200,  0.0193,  0.0186,  ...,  0.0403,  0.0404,  0.0404],
          [ 0.0214,  0.0210,  0.0206,  ...,  0.0416,  0.0419,  0.0423],
          [ 0.0226,  0.0226,  0.0226,  ...,  0.0429,  0.0435,  0.0441],
          ...,
          [ 0.0358,  0.0351,  0.0344,  ...,  0.0226,  0.0238,  0.0249],
          [ 0.0361,  0.0355,  0.0349,  ...,  0.0224,  0.0233,  0.0243],
          [ 0.0363,  0.0358,  0.0354,  ...,  0.0222,  0.0229,  0.0238]],

         [[ 0.1012,  0.1006,  0.1001,  ...,  0.1081,  0.1080,  0.1078],
          [ 0.1007,  0.1005,  

In [15]:
out_logit.shape, emb.shape

((1024, 543, 13), torch.Size([1, 512, 1376, 736]))

In [17]:
emb

tensor([[[[ 0.0174,  0.0173,  0.0172,  ...,  0.0073,  0.0070,  0.0067],
          [ 0.0181,  0.0179,  0.0177,  ...,  0.0079,  0.0076,  0.0072],
          [ 0.0187,  0.0185,  0.0182,  ...,  0.0086,  0.0081,  0.0076],
          ...,
          [ 0.0175,  0.0172,  0.0167,  ...,  0.0128,  0.0129,  0.0129],
          [ 0.0178,  0.0175,  0.0172,  ...,  0.0126,  0.0126,  0.0126],
          [ 0.0180,  0.0178,  0.0177,  ...,  0.0125,  0.0124,  0.0123]],

         [[ 0.0200,  0.0193,  0.0186,  ...,  0.0403,  0.0404,  0.0404],
          [ 0.0214,  0.0210,  0.0206,  ...,  0.0416,  0.0419,  0.0423],
          [ 0.0226,  0.0226,  0.0226,  ...,  0.0429,  0.0435,  0.0441],
          ...,
          [ 0.0358,  0.0351,  0.0344,  ...,  0.0226,  0.0238,  0.0249],
          [ 0.0361,  0.0355,  0.0349,  ...,  0.0224,  0.0233,  0.0243],
          [ 0.0363,  0.0358,  0.0354,  ...,  0.0222,  0.0229,  0.0238]],

         [[ 0.1012,  0.1006,  0.1001,  ...,  0.1081,  0.1080,  0.1078],
          [ 0.1007,  0.1005,  

In [17]:
import numpy as np

In [18]:
id_to_label = {i: v for i, v in enumerate(user_label)}
id_to_label[255] = 'unlabel'

In [21]:
prediction.shape

(1024, 543)

In [20]:
prediction

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [19]:
prediction = out_logit.argmax(axis=-1).squeeze()   # (h, w)
probs = out_logit.max(axis=-1).squeeze()  # (h, w)
high_prob_mask = probs > 0.5

mask = high_prob_mask
prediction[~mask] = 255

labels = np.unique(prediction)
label_names = {i: id_to_label[i] for i in labels}
# label_names = [id_to_label[i] for i in labels]
print()
print(f'label_names for img', label_names)


label_names for img {1: 'facade', 6: 'balcony', 10: 'pillar', 11: 'shop', 255: 'unlabel'}


In [30]:
id_to_label

{0: 'background',
 1: 'facade',
 2: 'window',
 3: 'door',
 4: 'cornice',
 5: 'sill',
 6: 'balcony',
 7: 'blind',
 8: 'deco',
 9: 'molding',
 10: 'pillar',
 11: 'shop',
 12: 'unlabel',
 255: 'unlabel'}

In [28]:
prediction.shape

(1024, 543)